# import

In [ ]:
from minone import CD, model, trainer, tokenization

from tqdm import tqdm
from ast import literal_eval


import torch
from torch.utils.data import Dataset
import pandas as pd
import re
from transformers import BertModel, BertTokenizer, DistilBertModel


In [ ]:
import os

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
device

device(type='mps')

# 데이터 및 모델 로드

In [ ]:
gwa_df = pd.read_csv('../data//0. 구_과_하는일_통합.csv')

In [ ]:
def extract_korean_text(text):
    if isinstance(text, str):
        korean_pattern = re.compile('[ㄱ-ㅎㅏ-ㅣ가-힣]+')
        korean_words = korean_pattern.findall(text)
        return ' '.join(korean_words)
    else:
        return ''

gwa_df['clean'] = gwa_df['하는일'].apply(extract_korean_text)

models = DistilBertModel.from_pretrained('monologg/distilkobert')
tokenizers = tokenization.KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
# 모델 생성
c_model = model.CustomModel(models, 768, 35)

# 미리 학습된 가중치 파일 경로
model_checkpoint = "./minone/model.pth"

# 모델 가중치 파일이 존재하는 경우에만 로드
if os.path.exists(model_checkpoint):
    parameter = torch.load(model_checkpoint, map_location=torch.device('cpu'))
    model_keys = set(c_model.state_dict().keys())
    pretrained_keys = set(parameter.keys())
    unexpected_keys = pretrained_keys - model_keys

    trimmed_state_dict = {k: v for k, v in parameter.items() if k not in unexpected_keys}

    c_model.load_state_dict(trimmed_state_dict, strict=False)
    print("모델 가중치를 성공적으로 로드했습니다.")
else:
    print("미리 학습된 가중치 파일이 없어 기본 모델을 사용합니다.")

모델 가중치를 성공적으로 로드했습니다.


# 임베딩 추출

In [ ]:
def inference(model, dataloader, device):
    model.to(device)
    model.eval()

    embeddings = []
    predictions = []

    with torch.no_grad():
        for input_ids, attention_mask in dataloader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            cls_output, pred = model(input_ids, attention_mask)
            predictions.extend(pred.argmax(dim=1).cpu().tolist())

            embeddings.append(cls_output)

    # 모든 임베딩을 하나의 텐서로 연결
    embeddings = torch.cat(embeddings, dim=0)

    return embeddings, predictions


In [ ]:
# 하는 일 임베딩
FeatureEmbedding = CD.FeatureExtractDataset(gwa_df['clean'].values, tokenizers)
FeatureEmbedding_dataloader = torch.utils.data.DataLoader(FeatureEmbedding, batch_size = 1, shuffle=False)
cls_output, _ = inference(c_model, FeatureEmbedding_dataloader, device)

In [ ]:
# cls_output을 numpy array로 변환
embeddings_np = cls_output.cpu().numpy()

# numpy array를 list of arrays로 변환
embedding_list = [embeddings_np[i] for i in range(embeddings_np.shape[0])]

# DataFrame에 새로운 열로 추가
gwa_df['embedding'] = embedding_list


In [ ]:
data = gwa_df.drop(['하는일',	'ex_token',	'ex_text',	'token'	,'text'], axis=1)

In [ ]:
# 임베딩이 텍스트 형태로 나타나는 것에 대한 수정 코드
data['embedding'] = data['embedding'].apply(lambda x: list(map(float, x)))

# 문자열을 리스트로 변환 후, 각 원소를 float로 변환
data['embedding'] = data['embedding'].apply(lambda x: list(map(float, literal_eval(x))))


In [ ]:
data.to_csv('../data/gwa_embedding.csv', index_col=0)